In [1]:
import numpy as np

model=np.load('/home/charles/Workspace/contour_detection/ContourGAN/models/vgg16_weights.npz')
print("model:", model)

model: <numpy.lib.npyio.NpzFile object at 0x7f60740abdd8>


In [2]:
vgg=np.load('/home/charles/Workspace/contour_detection/ContourGAN/models/vgg16.npy', encoding='bytes')
# vgg

In [5]:
print(model['conv1_1_b'])

[ 0.73429835  0.09340367  0.06775674  0.8862966   0.25994542  0.66426694
 -0.01582893  0.3249065   0.68600726  0.06247932  0.58156496  0.2361475
  0.69694996  0.19451167  0.4858922   0.44571847  0.5113422   0.208576
  0.57557714  0.33199573  0.4997983   0.7117759   0.30284074  0.7082712
  0.04548979  0.7446502   0.29845494  0.48211655  0.81658626  0.62603897
  0.3768093   2.064037    0.77311045  0.3459577   0.6130958   0.65459156
  0.39045632  0.50869167  0.2625384   0.23669638  0.07971057  1.1179353
  0.26129362  0.8697589   0.21543622  0.78007823  0.37015367  0.47993386
  0.4313978   0.5084194   0.23049663  0.7636527   0.35419866  0.45794216
  0.4662595   0.09850298  0.3803252   0.66880196  0.4015123   0.90510356
  0.43166816  1.302014    0.5306885   0.48993504]


In [4]:
for i,x in enumerate(model['params'][:26]):
    print("i:", i)
    print("x:", x)

KeyError: 'params is not a file in the archive'

data

['/home/charles/data/PASCALContourData/JPEGImages/2009_001514.jpg', '/home/charles/data/PASCALContourData/JPEGImages/2009_004999.jpg', '/home/charles/data/PASCALContourData/JPEGImages/2007_006136.jpg', ...]

label

['/home/charles/data/PASCALContourData/SegmentationObjectFilledDenseCRF/2010_000687.png', '/home/charles/data/PASCALContourData/SegmentationObjectFilledDenseCRF/2008_000338.png', '/home/charles/data/PASCALContourData/SegmentationObjectFilledDenseCRF/2010_004501.png' ...]

In [46]:
# read data
import os
from glob import glob
import numpy as np

data_dir='/home/charles/data/PASCALContourData/JPEGImages/'
label_dir='/home/charles/data/PASCALContourData/SegmentationObjectFilledDenseCRF'

# data_dir = '/home/charles/data/HED-BSDS'
# label_dir = '/home/charles/data/HED-BSDS'

input_fname_pattern='*.jpg'
label_fname_pattern='*.png'
data_path = os.path.join(data_dir, input_fname_pattern)
label_path = os.path.join(label_dir, label_fname_pattern)
# use 'glob' to search specific files. ex. '/data/BSDS/*.jpg'
# data: list of input images. ex. ['path/to/image/xx.jpg', ...,path/to/image/xxy.jpg]
data = glob(data_path)
label = glob(label_path)

np.random.shuffle(data)
# print("data:", data)
# print("label:", label)
batch_size = 16
print("[INFo] data length:", len(data))
print("[INFo] train size:", np.inf)

batch_idxs = min(len(data), np.inf) // batch_size
print("[INFO] batch_idxs:",batch_idxs )

[INFo] data length: 17125
[INFo] train size: inf
[INFO] batch_idxs: 1070


In [51]:
import cv2
import scipy.misc

def imread(path, grayscale=False):
    if (grayscale):
        return scipy.misc.imread(path, flatten=True).astype(np.float)
    else:
        # Reference: https://github.com/carpedm20/DCGAN-tensorflow/issues/162#issuecomment-315519747
        img_bgr = cv2.imread(path)
        # Reference: https://stackoverflow.com/a/15074748/
        img_rgb = img_bgr[..., ::-1]
        return img_rgb.astype(np.float)

def center_crop(x, crop_h, crop_w,
                resize_h=64, resize_w=64):
    if crop_w is None:
        crop_w = crop_h
    h, w = x.shape[:2]
    j = int(round((h - crop_h) / 2.))
    i = int(round((w - crop_w) / 2.))
    return scipy.misc.imresize(
        x[j:j + crop_h, i:i + crop_w], [resize_h, resize_w])

def transform(image, input_height, input_width,
              resize_height=64, resize_width=64, crop=True):
    if crop:
        cropped_image = center_crop(
            image, input_height, input_width,
            resize_height, resize_width)
    else:
        cropped_image = scipy.misc.imresize(image, [resize_height, resize_width])
    return np.array(cropped_image) / 127.5 - 1.


def get_image(image_path, input_height, input_width,
              resize_height=64, resize_width=64,
              crop=True, grayscale=False):
    image = imread(image_path, grayscale)
    return transform(image, input_height, input_width,
                     resize_height, resize_width, crop)



In [58]:
output_height = 480
output_width = 480
crop = True
grayscale=False
"""
You are trying to run a Python 2 codebase with Python 3. 
xrange() was renamed to range() in Python 3.
"""
for idx in range(0, int(batch_idxs)):
    batch_files = data[idx*batch_size:(idx+1)*batch_size]
    batch = [get_image(batch_file,
            input_height=480,
            input_width=480,
            resize_height=output_height,
            resize_width=output_width,
            crop=crop,
            grayscale=grayscale) for batch_file in batch_files]
    #print("[INFO] batch:", batch)

/home/charles/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [19]:
data_dir='/home/charles/data/PASCALContourData/'
f1_path = data_dir + 'train.txt'
f2_path = data_dir + 'train_modified.txt'


#Filename: file_read_and_write.py
#打开文件，cNames读取所有行，储存在列表中，循环对每一行在起始处加上序号1，2，3，4
with open(r'/home/charles/data/PASCALContourData/train.txt') as f1:
    cNames = f1.readlines()
    for i in range(0,len(cNames)):
        # cNames[i] = str(i+1) + '.' + '' + cNames[i]
        cNames[i] = 'JPEGImages/' + '' + cNames[i]
#将处理过的cNames写入新的文件中
with open(r'/home/charles/data/PASCALContourData/train2.txt','w') as f2:
    f2.writelines(cNames)


In [20]:
with open(r'/home/charles/data/PASCALContourData/val.txt') as f1:
    cNames = f1.readlines()
    for i in range(0,len(cNames)):
        # cNames[i] = str(i+1) + '.' + '' + cNames[i]
        cNames[i] = 'SegmentationObjectFilledDenseCRF/' + '' + cNames[i]
#将处理过的cNames写入新的文件中
with open(r'/home/charles/data/PASCALContourData/val2.txt','w') as f2:
    f2.writelines(cNames)

Python strip() 方法用于移除字符串头尾指定的字符（默认为空格）或字符序列。
The strip() removes characters from both left and right based on the argument (a string specifying the set of characters to be removed).

str = "00000003210Runoob01230000000"; 

print str.strip( '0' );  # 去除首尾字符 0

str2 = "   Runoob      ";   # 去除首尾空格

print str2.strip();

以上实例输出结果如下：

3210Runoob0123

Runoob

For ex, in read_file_list()

file before strip

[ INFO 0]
[ 'train/aug_data/0.0_1_0/100075.jpg train/aug_gt/0.0_1_0/100075.png\n', 'train/aug_data/0.0_1_0/100080.jpg train/aug_gt/0.0_1_0/100080.png\n', ... ]

file after strip

[ INFO 1]
['train/aug_data/0.0_1_0/100075.jpg train/aug_gt/0.0_1_0/100075.png', 'train/aug_data/0.0_1_0/100080.jpg train/aug_gt/0.0_1_0/100080.png', ..]

For ex, in split_par_name()

[ INFO 0]
[['train/aug_data/0.0_1_0/100075.jpg', 'train/aug_gt/0.0_1_0/100075.png'], ['train/aug_data/0.0_1_0/100080.jpg', 'train/aug_gt/0.0_1_0/100080.png'], ['train/aug_data/0.0_1_0/100098.jpg', 'train/aug_gt/0.0_1_0/100098.png'], ['train/aug_data/0.0_1_0/101085.jpg', 'train/aug_gt/0.0_1_0/101085.png'], ['train/aug_data/0.0_1_0/101087.jpg', 'train/aug_gt/0.0_1_0/101087.png'], ...]

[ INFO 1]
[('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/90.0_1_1/94079.jpg',
'/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/90.0_1_1/94079.png'), 
('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/90.0_1_1/95006.jpg', 
'/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/90.0_1_1/95006.png'),
('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/90.0_1_1/97017.jpg', 
'/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/90.0_1_1/97017.png'),
('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/90.0_1_1/97033.jpg',
'/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/90.0_1_1/97033.png'), ...]


In [1]:
import cv2

def read_file_list(filelist):
    # open file
    pfile = open(filelist)
    # read file line by line 
    filenames = pfile.readlines()
    # print("[INFO 0] filenames:", filenames)
    pfile.close()
    filenames = [f.strip() for f in filenames]
    # print("[INFO 1] filenames:", filenames)
    
    return filenames

def split_pair_names(filenames, base_dir):
    # splite elements by space (' ')
    filenames = [c.split(' ') for c in filenames]
    # print("[ INFO 0 ] ",filenames )
    filenames = [(os.path.join(base_dir, c[0]), os.path.join(base_dir, c[1])) for c in filenames]
    # print("[ INFO 1 ] ",filenames )
    return filenames

def imread(path, grayscale=False):
    if (grayscale):
        return scipy.misc.imread(path, flatten=True).astype(np.float)
    else:
        # Reference: https://github.com/carpedm20/DCGAN-tensorflow/issues/162#issuecomment-315519747
        img_bgr = cv2.imread(path)
        # Reference: https://stackoverflow.com/a/15074748/
        img_rgb = img_bgr[..., ::-1]
        return img_rgb.astype(np.float)

In [2]:
import os
import numpy as np

train_file = '/home/charles/data/HED-BSDS/train_pair.lst'
train_data_dir = '/home/charles/data/HED-BSDS'

print('[INFO] Training data set-up from {}'.format(os.path.join(train_file)))

training_pairs = read_file_list(train_file)
print("[INFO] training_pair[0]", training_pairs[0])
samples = split_pair_names(training_pairs, train_data_dir)
print("[INFO] samples[0]", samples[0])
print("[INFO] samples[0][0]", samples[0][0])
print("[INFO] samples[0][1]", samples[0][1])

imreadImg = imread(samples[0][0])
# check if image is a non-gray-scale image by checking channel number
if len(imreadImg.shape) >= 3:
    c_dim = imread(samples[0][0]).shape[-1]
else:
    c_dim = 1

print("[INFO] imreadImg:", imreadImg.shape)
print("[INFO] c_dim:", c_dim)

imreadLabel = imread(samples[0][1])
# check if image is a non-gray-scale image by checking channel number
if len(imreadLabel.shape) >= 3:
    c_dim_label = imread(samples[0][1]).shape[-1]
else:
    c_dim_label = 1
    
print("[INFO] imreadLabel:", imreadLabel.shape)
print("[INFO] c_dim_label:", c_dim_label)

n_samples = len(training_pairs)
print("[INFO] n_samples", n_samples)
all_ids = range(n_samples)
# all_ids: range(0, 28800)
print("[INFO] all_ids:", all_ids)
# convert 'range' object to a 'list'
# all_ids = list(range(len(all_ids)))
# print("[INFO] all_ids:", all_ids)
"""
TypeError: 'range' object does not support item assignment
In Python 3, range returns a lazy sequence object - it does not return a list.
There is no way to rearrange elements in a range object, so it cannot be shuffled.
Convert it to a list before shuffling.
allocations = list(range(len(people)))
"""
all_ids = list(range(n_samples))
# print("[INFO] all_ids:", all_ids)

# 打乱 list 的顺序
np.random.shuffle(all_ids)
# print("[INFO] all_ids:", all_ids)
# train set = 0.8, val set = 0.2.
train_split = 0.8
print("[INFO] train pair length:", len(training_pairs))
training_ids = all_ids[:int(train_split * len(training_pairs))]
validation_ids = all_ids[int(train_split * len(training_pairs)):]

# index for train set
# print("[INFO] training_ids:", training_ids)
# print("[INFO] validation_ids:", validation_ids)

print('[INFO] Training samples {}'.format(len(training_ids)))
print('[INFO] Validation samples {}'.format(len(validation_ids)))

[INFO] Training data set-up from /home/charles/data/HED-BSDS/train_pair.lst
[INFO] training_pair[0] train/aug_data/0.0_1_0/100075.jpg train/aug_gt/0.0_1_0/100075.png
[INFO] samples[0] ('/home/charles/data/HED-BSDS/train/aug_data/0.0_1_0/100075.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/0.0_1_0/100075.png')
[INFO] samples[0][0] /home/charles/data/HED-BSDS/train/aug_data/0.0_1_0/100075.jpg
[INFO] samples[0][1] /home/charles/data/HED-BSDS/train/aug_gt/0.0_1_0/100075.png
[INFO] imreadImg: (384, 544, 3)
[INFO] c_dim: 3
[INFO] imreadLabel: (384, 544, 3)
[INFO] c_dim_label: 3
[INFO] n_samples 28800
[INFO] all_ids: range(0, 28800)
[INFO] train pair length: 28800
[INFO] Training samples 23040
[INFO] Validation samples 5760


convert RGB image to Grayscle image

from PIL import Image

I = Image.open(samples[0][1])

I.show()

L = I.convert('L')

L.show()

training_ids: length = 23040

[28544, 15484, 19370, 5493, 3156, 28480, 15696, 24874, 17172, 27811, 9228, 8147, 6178, 986, 26693, 589, 3765, 10405, 12995, 16009, 14537, 17657, 10582, 9122, 11342, 2574, 10400, 11555, 12529, 7551, 1943, 7750, 28534, 10958, 13256, 2722, 24501, 10648, 8164, 1310, ...]

validation_ids: length = 5760

[19526, 6306, 18188, 19944, 26370, 9554, 22254, 18384, 22929, 5244, 9620, 25025, 7914, 26099, 28784, 22864, 19128, 5804, 17055, 3954, 1079, 20362, 15191, 26857, 1571, 12097, 23001, 14030, 7313, 23173, 27119, 12854, 15425, 18017, 5877, 23316, 19090,  ...]


In [11]:
from PIL import Image
import time 

def get_training_batch(training_ids, batch_size_train):
    # 从　train_ids 中随机选择　batch size 个　indexs
    batch_ids = np.random.choice(training_ids, batch_size_train)
    print("[INFO] batch_ids traing set:", batch_ids)
    return get_batch(batch_ids)

def get_validation_batch(validation_ids, batch_size_val):

    batch_ids = np.random.choice(validation_ids, batch_size_val)
    print("[INFO] batch_ids val set:", batch_ids)
    # 调用　batch_size　次 get_batch()
    return get_batch(batch_ids)

def get_batch(batch, samples=samples, image_width_train=480, image_height_train=480):
    # RGB to BGR (VGG-16 trained with BGR blame OpenCV)
    channel_swap = [2, 1, 0]

    # Mean pixel value to subtract from BGR image
    mean_pixel_value = [103.939, 116.779, 123.68]
    
    # Targets are continous if True else binary {0, 1}
    target_regression = True
    
    tstart = time.time()

    filenames = []
    images = []
    edgemaps = []

    for idx, b in enumerate(batch):
            print("[INFO] idx of batch", idx)
            print("[INFO] b of batch", b)
            
            # input image 
            im = Image.open(samples[b][0])
            print("[INFO] samples[b][0]:", samples[b][0])
            # edge map
            em = Image.open(samples[b][1])
            print("[INFO] samples[b][1]:", samples[b][1])

            im = im.resize((image_width_train, image_height_train))
            em = em.resize((image_width_train, image_height_train))

            im = np.array(im, dtype=np.float32)
            im = im[:, :, channel_swap]
            im -= mean_pixel_value

            # Labels needs to be 1 or 0 (edge pixel or not)
            # or can use regression targets as done by the author
            # https://github.com/s9xie/hed/blob/9e74dd710773d8d8a469ad905c76f4a7fa08f945/src/caffe/layers/image_labelmap_data_layer.cpp#L213

            em = np.array(em.convert('L'), dtype=np.float32)

            if target_regression:
                bin_em = em / 255.0
            else:
                bin_em = np.zeros_like(em)
                bin_em[np.where(em)] = 1

            # Some edge maps have 3 channels some dont
            bin_em = bin_em if bin_em.ndim == 2 else bin_em[:, :, 0]
            # To fit [batch_size, H, W, 1] output of the network
            bin_em = np.expand_dims(bin_em, 2)

            images.append(im)
            print("[INFO] ", len(images))
            edgemaps.append(bin_em)
            print("[INFO] ", len(edgemaps))
            print("[INFO] ", len(edgemaps))
            filenames.append(samples[b])
            print("[INFO] filenames length:", len(filenames))
            print("[INFO] filenames:", filenames)

    return images, edgemaps, filenames

samples[b=23974]

samples[b]: 

[('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/22.5_1_1/68077.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/22.5_1_1/68077.png')]

samples[b][0]: 

/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/22.5_1_1/68077.jpg
 
samples[b][1]: 

/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/22.5_1_1/68077.png


In [13]:
batch_size = 16
batch_size_val =  16
batch_idxs = min(len(training_ids), np.inf) // batch_size
print("[INFO] batch_ids:", batch_idxs)
batch_images_list, batch_z_list, _ = get_training_batch(training_ids, batch_size_train)

[INFO] batch_ids: 1440
[INFO] batch_ids traing set: [ 5486  4932 21681 13330 26829 16830  9756  3651 15235 11045  9089 19786
 10549  6849  5750 28536]
[INFO] idx of batch 0
[INFO] b of batch 5486
[INFO] samples[b][0]: /home/charles/data/HED-BSDS/train/aug_data/270.0_1_0/170054.jpg
[INFO] samples[b][1]: /home/charles/data/HED-BSDS/train/aug_gt/270.0_1_0/170054.png
[INFO]  1
[INFO]  1
[INFO]  1
[INFO] filenames length: 1
[INFO] filenames: [('/home/charles/data/HED-BSDS/train/aug_data/270.0_1_0/170054.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/270.0_1_0/170054.png')]
[INFO] idx of batch 1
[INFO] b of batch 4932
[INFO] samples[b][0]: /home/charles/data/HED-BSDS/train/aug_data/247.5_1_0/219090.jpg
[INFO] samples[b][1]: /home/charles/data/HED-BSDS/train/aug_gt/247.5_1_0/219090.png
[INFO]  2
[INFO]  2
[INFO]  2
[INFO] filenames length: 2
[INFO] filenames: [('/home/charles/data/HED-BSDS/train/aug_data/270.0_1_0/170054.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/270.0_1_0/170054.png'),

[INFO] samples[b][1]: /home/charles/data/HED-BSDS/train/aug_gt/90.0_1_0/173036.png
[INFO]  11
[INFO]  11
[INFO]  11
[INFO] filenames length: 11
[INFO] filenames: [('/home/charles/data/HED-BSDS/train/aug_data/270.0_1_0/170054.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/270.0_1_0/170054.png'), ('/home/charles/data/HED-BSDS/train/aug_data/247.5_1_0/219090.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/247.5_1_0/219090.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/180.0_1_0/164074.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/180.0_1_0/164074.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/225.0_1_0/216066.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/225.0_1_0/216066.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/337.5_1_1/216053.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/337.5_1_1/216053.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/337.5_1_0/12003.jpg', '/home/charles/data/HED-BSDS/t

[INFO] samples[b][0]: /home/charles/data/HED-BSDS/train/aug_data/270.0_1_1/143090.jpg
[INFO] samples[b][1]: /home/charles/data/HED-BSDS/train/aug_gt/270.0_1_1/143090.png
[INFO]  15
[INFO]  15
[INFO]  15
[INFO] filenames length: 15
[INFO] filenames: [('/home/charles/data/HED-BSDS/train/aug_data/270.0_1_0/170054.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/270.0_1_0/170054.png'), ('/home/charles/data/HED-BSDS/train/aug_data/247.5_1_0/219090.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/247.5_1_0/219090.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/180.0_1_0/164074.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/180.0_1_0/164074.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/225.0_1_0/216066.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/225.0_1_0/216066.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/337.5_1_1/216053.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/337.5_1_1/216053.png'), ('/home/charles/data

[INFO] batch_ids traing set: 

[23974     0   582 25814 16538 18797  1696 10390 11890 15691  2605 20361
 18083 16732 22712  6525]
 
[INFO] idx of batch 0

[INFO] b of batch 23974

[INFO]  1

[INFO]  1

[INFO]  1

[INFO] filenames length: 1

[INFO] filenames: 

[('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/22.5_1_1/68077.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/22.5_1_1/68077.png')]

[INFO] idx of batch 15

[INFO] b of batch 6525

[INFO]  16

[INFO]  16

[INFO]  16

[INFO] filenames length: 16

[INFO] filenames: [('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/22.5_1_1/68077.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/22.5_1_1/68077.png'), ('/home/charles/data/HED-BSDS/train/aug_data/0.0_1_0/100075.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/0.0_1_0/100075.png'), ('/home/charles/data/HED-BSDS/train/aug_data/0.0_1_1/78019.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/0.0_1_1/78019.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/315.0_1_0/106025.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/315.0_1_0/106025.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/315.0_1_1/130026.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/315.0_1_1/130026.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/90.0_1_0/302008.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/90.0_1_0/302008.png'), ('/home/charles/data/HED-BSDS/train/aug_data/135.0_1_1/302003.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/135.0_1_1/302003.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/112.5_1_0/296007.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/112.5_1_0/296007.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/157.5_1_1/296007.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/157.5_1_1/296007.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/292.5_1_0/175043.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/292.5_1_0/175043.png'), ('/home/charles/data/HED-BSDS/train/aug_data/180.0_1_0/311081.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/180.0_1_0/311081.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/112.5_1_1/60079.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/112.5_1_1/60079.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/67.5_1_0/167062.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/67.5_1_0/167062.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_0.5/315.0_1_1/376043.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_0.5/315.0_1_1/376043.png'), ('/home/charles/data/HED-BSDS/train/aug_data_scale_1.5/202.5_1_1/35008.jpg', '/home/charles/data/HED-BSDS/train/aug_gt_scale_1.5/202.5_1_1/35008.png'), ('/home/charles/data/HED-BSDS/train/aug_data/292.5_1_1/370036.jpg', '/home/charles/data/HED-BSDS/train/aug_gt/292.5_1_1/370036.png')]

In [14]:
"""
for idx in range(int(batch_idxs)):
    print("idx", idx)
"""    
print("batch_images_list:", len(batch_images_list))
print("batch_z_list:", len(batch_z_list))

im: 16
em: 16


In [17]:
cv2.imread(samples[0][1])

# batch_images_raws = [cv2.imread(imgs) for imgs in batch_images_list]

for imgs in batch_images_list:
    batch_images_raws += [cv2.imread(imgs)]
    
batch_images = np.array(batch_images_raws)

batch_z_raws = [cv2.imread(imgs, 0) for imgs in batch_z_list]
batch_z = np.array(batch_z_raws)

print("[INFO] batch_images:", batch_images.shape)
print("[INFo] batch_z:", batch_z.shape)
print("[INFO] self.labels", self.labels)

NameError: name 'batch_images_raws' is not defined